In [ ]:
import mdsine2 as md2
import pandas as pd
import numpy as np

# set negative binomial model inference parameters
params = md2.config.NegBinConfig(
  #seed=0, burnin=100, n_samples=200,
  seed=0, burnin=10, n_samples=20,
  checkpoint=10, 
  basepath=str("./negbin")
)

dataset = md2.dataset.parse(
  name="david-diet", 
  taxonomy="taxonomy.tsv",  
  reads="reads.tsv", 
  qpcr="qpcr.tsv", 
  metadata="metadata.tsv",
  perturbations="perturbations.tsv"
)

In [ ]:
mcmc_negbin = md2.negbin.build_graph(
  params=params, 
  graph_name=dataset.name,
  subjset=dataset
)

In [ ]:
mcmc_negbin.run()

In [ ]:
from mdsine2.names import STRNAMES

print('a0', md2.summary(mcmc_negbin.graph[STRNAMES.NEGBIN_A0]))
print('a1', md2.summary(mcmc_negbin.graph[STRNAMES.NEGBIN_A1]))

In [ ]:
import pathlib

a0 = md2.summary(mcmc_negbin.graph[STRNAMES.NEGBIN_A0])['mean']
a1 = md2.summary(mcmc_negbin.graph[STRNAMES.NEGBIN_A1])['mean']

basepath = pathlib.Path("./mdsine2/david")
basepath.mkdir(exist_ok=True, parents=True)

# Initialize parameters of the model (Seed = 0) burnin=50, total steps=100
params = md2.config.MDSINE2ModelConfig(
    basepath=str(basepath), 
    seed=0,
    burnin=50, 
    n_samples=100, 
    negbin_a0=a0, negbin_a1=a1, 
    checkpoint=50
)

mcmc = md2.initialize_graph(params=params, graph_name=dataset.name, subjset=dataset)
mcmc = md2.run_graph(mcmc, crash_if_error=True)

https://github.com/gerberlab/MDSINE2#computation-and-visualization-of-forward-simulation-of-learned-glv-model-for-each-otu

In [ ]:
dataset._subjects["Plant5"].perturbations.__dict__["_d"]["Plant"].starts

In [ ]:
pd.read_csv("perturbations.tsv", sep = "\t")